In [ ]:
import numpy as np
import matplotlib.pyplot as plt

しかし、DMD (Dynamic Mode Decomposition) ってなんだ？

※ここで取り扱う全ての行列は、要素が実数です。複素数にも適用できますが、ここでは説明しません。

# 前知識

## (L2 / フロベニウス)ノルム

ベクトルや行列の大きさを調べる指標。言葉で言うと「要素の二乗和の平方」。式で言うと、

- L2ノルム: $X \in \R^N$ について、 $\| X \|_2 = \sqrt{\sum_{n=0}^N x_n^2}$
- フロベニウスノルム: $X \in \R^{M \times N}$ について、$\| X \|_F = \sqrt{\sum_{m=0}^M\sum_{n=0}^N x_{mn}^2}$

二つは対象が違うだけで言っていることは同じである。

In [ ]:
# L2ノルム
x_array = np.array([1, -1, 2])

# 手作業
print(np.sqrt(np.sum(x_array**2)))
# 組み込み関数
print(np.linalg.norm(x_array))

# フロベニウスノルム
x_mat = np.array(
  [
    [0, -1, 0],
    [1, 0, 2],
    [0, -1, 0]
  ]
)

# 手作業
print(np.sqrt(np.sum(x_mat**2)))
# 組み込み関数
print(np.linalg.norm(x_mat))

## 特異値分解 (SVD分解)

ある任意の行列を $A = U\Sigma V$ の形に分解すること。ただし、
- UやVは正方行列かつ直交行列 ($UU^T=I, VV^T=I$)
- $\Sigma$ は左上から右下に向かって特異値を並べた行列(他位置は0)であり、サイズは $A$ に等しい

$AA^T$ か $A^TA$ の固有値分解ができたならば、特異値分解を計算できたことになる。($U$と$V$は対応しているため、片方が決まったら残りは固有値分解でない方法で解く必要がある)
- $A^TA = (U\Sigma V)^T (U\Sigma V) = V^T\Sigma^T U^T (U\Sigma V) = V^T\Sigma^T\Sigma V$ より、$(A^TA)V^T = V^T(\Sigma^T\Sigma) $。つまり、固有値行列=$\Sigma^T\Sigma$、固有ベクトル行列=$V^T$
- $AA^T = (U\Sigma V)(U\Sigma V)^T = (U\Sigma V)V^T\Sigma^T U^T = U\Sigma\Sigma^TU^T$ より、$(AA^T)U = U(\Sigma\Sigma^T) $。つまり、固有値行列=$\Sigma\Sigma^T$、固有ベクトル行列=$U$

なお、$U, V$の組は複数ありうるが、$\Sigma$ はただ一つしかない。

補足：固有値分解の方法で$U$か$V$が求まったら、残りは$A = U\Sigma V$に立ち返って求める。今回はVが求まったとして説明。

> $A = U\Sigma V$  
> $AV^T = U\Sigma $

と変形できるので、結局

> $U = AV^T\Sigma^{-1} $

となる。ここで、
- $\Sigma^{-1}$ は $\Sigma$ の各非ゼロ要素の逆数を取り、転置したもの

In [ ]:
# SVD分解
# 入力
a = np.random.randn(3, 5)
print("a:")
print(a)

# 手作業で計算してみる
sigma_sq, v_t = np.linalg.eig(a.T @ a)
sort_index = np.argsort(-np.abs(sigma_sq))
sigma_sq = sigma_sq[sort_index]
v_t = v_t[:, sort_index]
# 正しく分解できたかどうか
print(np.linalg.norm(a.T @ a @ v_t - v_t @ np.diag(sigma_sq)))

v = v_t.T
sigma = np.zeros_like(a, dtype=float)
np.fill_diagonal(sigma, np.sqrt(sigma_sq))

u = a @ v_t @ np.where(sigma != 0, 1 / sigma, 0).T

print("\nu:")
print(u)

print("\nsigma:")
print(sigma)

print("\nv:")
print(v)

print("\nu sigma v (= a):")
print(u @ sigma @ v)

# 組み込み関数
u, s, v = np.linalg.svd(a)
sigma = np.zeros_like(a, dtype=float)
np.fill_diagonal(sigma, s)

print("\nu:")
print(u)

print("\nsigma:")
print(sigma)

print("\nv:")
print(v)

print("\nu sigma v (= a):")
print(u @ sigma @ v)


## ムーア・ペンローズの疑似逆行列

逆行列は正方行列であってかつ正則でなければ定義されない。何とかそんな行列以外でも逆行列のようなものを定義したい。  
そのような時に使えるのが疑似逆行列。

ある $m \times n$ 行列$A \in \R^{m \times n}$について、以下を満たす$n \times m$行列 $A^+ \in \R^{n \times m}$ がただ一つ存在し、$A^+$ を疑似逆行列と呼ぶ。

> $A^+AA^+ = A^+$   
> $AA^+A = A$  
> $(A^+A)^T = (A^+A)$  
> $(AA^+)^T = (AA^+)$  

疑似逆行列は、特異値分解を用いて求めることができる。

$A = U\Sigma V$ より、

> $A^+ = V^T \Sigma^{-1} U^T$

で求められる

In [ ]:
a = np.random.randn(3, 5)
print("a:")
print(a)

u, s, v = np.linalg.svd(a)
sigma = np.zeros_like(a, dtype=float)
np.fill_diagonal(sigma, s)

# ムーア・ペンローズの逆行列
a_inv = v.T @ np.where(sigma != 0, 1 / sigma, 0).T @ u.T

print("\na_inv:")
print(a_inv)

print(np.allclose(a_inv @ a @ a_inv, a_inv))
print(np.allclose(a @ a_inv @ a, a))
print(np.allclose((a_inv @ a).T, a_inv @ a))
print(np.allclose((a @ a_inv).T, a @ a_inv))

# DMDが目指すところ

DMD は実際のデータから時空間パターンを抽出する事を目指す。

**ここでは実数の範囲で考えることにする。** (本来は複素数の範囲で適用可能)

さて、以下のデータ (スナップショットと呼ばれる) からパターンを抽出することを考える

> $X = \{x_1, ..., x_m | x \in \R^n\}$

ただし、
- $m$ はデータポイントの数
- $n$ は各データポイントのベクトルの要素数

要するに、$X$は $X \in \R^{n \times m}$ の行列である。

普通、$n >> m$ とすることが多い (小さめなスナップショットを取る)。が、別にこれに限らなくともよい。


**DMDの目標は、以下のように次のデータポイントを予測する行列 $A \in \R^{n \times n}$を見つけることである**

> $x_{j+1} \approx A x_{j}$

ただし、
- $j$ は整数 $(1 \le j \le m - 1)$

$A$ を求めるためには、一般に次の最適化問題を解く

> $\underset{A \in \R^{n \times n}}{min} \sum_{j=1}^{m-1} \|A x_j - x_{j+1}\|^2_2$

ただし、
- $\| ・ \|^2_2$ はL2ノルムの二乗 (つまり、要素の二乗和)

が、非常に面倒(計算コストがかかる)なので通常は近似して計算する

# 概略した方法

最適化問題を各$j$ について計算するのは面倒なので、行列にまとめる。
$x_j$ の集合を $X_1$とし、これを右に一つずらしたもの (つまり、$x_{j+1}$ の集合) を$X_2$ とする。つまり、

> $X_1 = \{x_1, x_2, ..., x_{m-1}\}$  
> $X_2 = \{x_2, x_3, ..., x_{m}\}$

とすると、問題は次のように書き直せる。

> $\underset{A \in \R^{n \times n}}{min} \|A X_1 - X_2\|^2_F$

ただし、
- $\| ・ \|^2_F$ はフロベニウスノルムの二乗 (つまり、要素の二乗和)

$X_1$ と $X_2$ は、ずらしただけなのでサイズが一致する $( X_1, X_2 \in \R^{n \times (m-1)})$。  


問題を端的に $AX_1 - X_2 \approx O$ と思い直すことにすると、$A$ は下式を計算すればよいことになる (厳密な証明はしない)

> $A = X_2 X_1^+$

ここで、$X_1^+$ は ムーア・ペンローズの疑似逆行列 である。

さて、例えば $k+1$ ステップ目のデータを計算するには、

>$x_{k+1} \approx A^{k}x_1$

のように計算できる。だが、$A^k$を計算するのは面倒。  

ここで、Aを対角化可能だと仮定することで計算を簡単にできる。つまり、$A$ を以下の式のように対角化する：
> $A = V\Lambda V^{-1}$

ただし、
- $V$ は対角化行列
  - 対角化行列であるということは、固有ベクトルを集めた行列でもあるということ
- $\Lambda$ は固有値を対角線上に持つ正方行列

こうすることで、$x_{k+1}$ は以下で表せる。  
> $x_{k+1} \approx A^{k}x_1 = (V\Lambda V^{-1})^k x_1 = V\Lambda^k V^{-1} x_1 = V\Lambda^k x_0$

ただし、
- $x_0 = V^{-1} x_1$ とおいた。$x_0$ は初項のようなものである。




ここまで発展させると、DMDの結果は $(\Lambda, V, x_0)$ を決定することに対応するといえる。それぞれ、DMD固有値、DMDモード、DMD振幅 と解釈される。

# SVD による決定方法

この問題を(近似的に)解く方法はいくつか提案されているが、ここではSVD分解を用いた方法を説明する。

データ $X$ のスライス $X_1$ が 
> $X_1=U\Sigma V$   

とSVD分解できるとする。このとき、それぞれの次元は、
- $X_1 \in \R^{n \times (m-1)}$
- $U \in \R^{n \times r}$  
- $\Sigma \in \R^{r \times r}$
- $V \in \R^{r \times m}$

となる。(rは自由に決め、SVD分解の結果をトリミングして決定する。これにより計算が簡略化する)

ここで、次のような$S$を決めると計算が簡単になる、というのがSVDによる方法である。

> $S= U^*AU \in \R^{r \times r}$

$S$を詳しく計算していくと、

> $S = U^*AU = U^*AU \Sigma V V^* \Sigma^{-1} = U^*A X_1 V^* \Sigma^{-1} = U^* X_2 V^* \Sigma^{-1}$

$S$ と $A$ の固有値は一致することが知られており、また、$S$ の固有ベクトルを$V'$ とすると、$V=UV$ として$A$の固有ベクトルを求められる (証明略)。したがって、$A$ の計算をせずとも $V$ や $\Lambda$ を知ることができる。



In [ ]:
# DMD 
r = 5
n = 100
m = 150

x = np.fromfunction(lambda n, m: 90 * np.sin(n * n * n + 3 * m), (n, m))
x1 = x[:, :-1]
x2 = x[:, 1:]
u, sigma, v = np.linalg.svd(x1)

u = u[:, :r]
sigma = np.diag(sigma[:r])
v = v[:r, :]

print("x1, x2: ",  x1.shape, x2.shape)
print("u, sigma, v:", u.shape, sigma.shape, v.shape)

s = np.linalg.pinv(u) @ \
    x2 @ \
    np.linalg.pinv(v) @ \
    np.linalg.inv(sigma)

lamb, vq = np.linalg.eig(s)
v = u @ vq
x0 = np.linalg.pinv(v) @ x[:, 0]
lamb = np.diag(lamb)

print(f"lambda: {lamb.shape}")
print(f"v: {v.shape}")
print(f"x0: {x0.shape}")

# これで、DMDにより \Lambda, V, X_0 が計算された。再構成誤差を測ってみる
def reconstract(k):
  return v @ np.where(lamb != 0, np.power(lamb, k), 0.0) @ x0

reconstracted = np.array([reconstract(i) for i in range(m)], dtype=float).T
print(f"再構成誤差: {np.linalg.norm(x - reconstracted)}")

for c in range(5):
  plt.plot(x[:,c])
  plt.plot(reconstracted[:, c])
  plt.show()


# 参考文献 (Webページは全て6/11に閲覧)

1. KRAKE, Tim; WEISKOPF, Daniel; EBERHARDT, Bernhard. Dynamic mode decomposition: Theory and data reconstruction. arXiv preprint arXiv:1909.10466, 2019.
2. https://qiita.com/harmegiddo/items/84552c32f4b75c26878a
3. https://manabitimes.jp/math/2746